# 판다스를 이용하여 데이터 조작하기
## 데이터 불러오기
한국도로공사 제공 데이터셋: [영업소간 통행시간](http://data.ex.co.kr/portal/fdwn/view?type=TCS&num=11&requestfrom=dataset)

In [1]:
import pandas as pd
file = './data/korea expressway corporation/TCS_11_03_02_339640.csv'
data = pd.read_csv(file, sep=',', encoding='euc-kr')

In [3]:
data.tail()

,집계일자,집계시,출발영업소코드,도착영업소코드,TCS차종구분코드,통행시간,Unnamed: 6
24767713,20210131,10,506,602,4,25473,NaN
24767714,20210131,14,172,215,3,1385,NaN
24767715,20210131,5,269,556,1,3115,NaN
24767716,20210131,5,500,101,1,2033,NaN
24767717,20210131,2,624,252,1,1896,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24767718 entries, 0 to 24767717
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   집계일자        int64  
 1   집계시         int64  
 2   출발영업소코드     int64  
 3   도착영업소코드     int64  
 4   TCS차종구분코드   int64  
 5   통행시간        int64  
 6   Unnamed: 6  float64
dtypes: float64(1), int64(6)
memory usage: 1.3 GB


## 데이터 정제
모든 값이 null인 `Unnamed: 6`를 제거하자.

In [5]:
data.isnull().sum(axis=0)

집계일자                 0
집계시                  0
출발영업소코드              0
도착영업소코드              0
TCS차종구분코드            0
통행시간                 0
Unnamed: 6    24767718
dtype: int64

In [6]:
data_clean = data.drop(['Unnamed: 6'], axis='columns')
data_clean.head()

,집계일자,집계시,출발영업소코드,도착영업소코드,TCS차종구분코드,통행시간
0,20210101,18,0,101,1,-1
1,20210101,18,0,101,3,-1
2,20210101,18,0,101,4,-1
3,20210101,9,0,101,3,-1
4,20210101,9,0,101,2,-1


통행시간이 0보다 작은 데이터를 제거하자.

In [7]:
(data_clean.통행시간 > 0).unique()

array([False,  True])

In [9]:
data_clean = data_clean[data_clean.통행시간 > 0]
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20817148 entries, 7684 to 24767717
Data columns (total 6 columns):
 #   Column     Dtype
---  ------     -----
 0   집계일자       int64
 1   집계시        int64
 2   출발영업소코드    int64
 3   도착영업소코드    int64
 4   TCS차종구분코드  int64
 5   통행시간       int64
dtypes: int64(6)
memory usage: 1.1 GB


차종구분코드는 사용하지 않는다.

In [10]:
df_data = pd.DataFrame(data_clean, columns=['집계일자', '집계시', '출발영업소코드', '도착영업소코드', '통행시간'])
df_data.head()

,집계일자,집계시,출발영업소코드,도착영업소코드,통행시간
7684,20210101,21,101,102,617
7686,20210101,21,101,102,601
7687,20210101,21,101,102,644
7688,20210101,21,101,102,600
7689,20210101,17,101,102,612


영업소 101에서 140으로 이동한 데이터를 분류해보자.

In [15]:
from_101 = df_data[df_data.출발영업소코드 == 101]
from_101_to_140 = from_101[from_101.도착영업소코드 == 140]
from_101_to_140.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 708 entries, 9157 to 24700511
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   집계일자     708 non-null    int64
 1   집계시      708 non-null    int64
 2   출발영업소코드  708 non-null    int64
 3   도착영업소코드  708 non-null    int64
 4   통행시간     708 non-null    int64
dtypes: int64(5)
memory usage: 33.2 KB


요일 데이터를 추가하자(0: 월, 1: 화, ..., 6: 일).

In [17]:
from_101_to_140['요일'] = pd.to_datetime(from_101_to_140['집계일자'], format='%Y%m%d').dt.dayofweek
from_101_to_140.head()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24096\3158426315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from_101_to_140['요일'] = pd.to_datetime(from_101_to_140['집계일자'], format='%Y%m%d').dt.dayofweek


,집계일자,집계시,출발영업소코드,도착영업소코드,통행시간,요일
9157,20210101,19,101,140,15924,4
9158,20210101,7,101,140,14237,4
9159,20210101,0,101,140,13556,4
9160,20210101,19,101,140,19430,4
9161,20210101,17,101,140,16025,4


정렬한다.

In [19]:
from_101_to_140.sort_values(by=['집계일자', '집계시'])

,집계일자,집계시,출발영업소코드,도착영업소코드,통행시간,요일
9159,20210101,0,101,140,13556,4
7542678,20210101,1,101,140,13595,4
1151591,20210101,2,101,140,13633,4
844398,20210101,3,101,140,13725,4
6964783,20210101,4,101,140,13796,4
...,...,...,...,...,...,...
23429292,20210131,19,101,140,15090,6
23429295,20210131,20,101,140,14940,6
23429289,20210131,21,101,140,15097,6
23429304,20210131,22,101,140,15547,6


In [23]:
groupby_destination = from_101['통행시간'].groupby(from_101['도착영업소코드'])
groupby_destination.size()

도착영업소코드
102    3420
103    4448
104     459
105    4242
106    4386
       ... 
762     679
840     193
850       2
851    1766
876    1973
Name: 통행시간, Length: 317, dtype: int64

In [24]:
groupby_destination.mean()

도착영업소코드
102      710.810234
103      635.478417
104    10682.501089
105      912.917020
106     1276.969904
           ...     
762     3945.537555
840     4483.849741
850     5863.000000
851     4164.588901
876     2201.002027
Name: 통행시간, Length: 317, dtype: float64

## 정리
결과를 저장한다.

In [25]:
output = './data/korea expressway corporation/save.csv'
from_101_to_140.to_csv(output, index=None, header=True)

결과를 다시 불러오자. 예제 처음에 사용한 구문이다.

In [28]:
data = pd.read_csv(output)
data.head()

,집계일자,집계시,출발영업소코드,도착영업소코드,통행시간,요일
0,20210101,19,101,140,15924,4
1,20210101,7,101,140,14237,4
2,20210101,0,101,140,13556,4
3,20210101,19,101,140,19430,4
4,20210101,17,101,140,16025,4


이런 데이터가 많다고 가정하고, 병합해보자.

In [29]:
data1 = pd.read_csv(output)
data2 = pd.read_csv(output)
data3 = pd.concat([data1, data2])
data3.head()

,집계일자,집계시,출발영업소코드,도착영업소코드,통행시간,요일
0,20210101,19,101,140,15924,4
1,20210101,7,101,140,14237,4
2,20210101,0,101,140,13556,4
3,20210101,19,101,140,19430,4
4,20210101,17,101,140,16025,4
